<a href="https://colab.research.google.com/github/srujayreddyv/CSUS-CSC219-MachineLeaning/blob/main/labs/CSC219Lab03_EvaluatingNeuralNetworks_Srujay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CSC 219 Machine Learning (Fall 2023)

#Lab3: Evaluating Neural Networks

##helper functions

In [ ]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


## 3.1 Training and Testing with early stopping

overfitting occurs when a neural network is trained to the point that it begins to memorize rather than generalize.

###Splitting data into training and testing using train_test_split

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

path = "/content/drive/MyDrive/Colab Notebooks/CSC219-MachineLearning/labs/data"

filename = os.path.join(path,"iris.csv")
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()

model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')

# patience: number of epochs with no improvement after which training will be stopped

# The test set is checked during training to monitor progress for early stopping but is never used for gradient descent (model training)

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor], verbose=2, epochs=1000)

Epoch 1/1000
4/4 - 2s - loss: 1.5745 - val_loss: 1.4326 - 2s/epoch - 461ms/step
Epoch 2/1000
4/4 - 0s - loss: 1.5092 - val_loss: 1.3825 - 53ms/epoch - 13ms/step
Epoch 3/1000
4/4 - 0s - loss: 1.4538 - val_loss: 1.3374 - 67ms/epoch - 17ms/step
Epoch 4/1000
4/4 - 0s - loss: 1.4029 - val_loss: 1.2984 - 66ms/epoch - 17ms/step
Epoch 5/1000
4/4 - 0s - loss: 1.3584 - val_loss: 1.2645 - 67ms/epoch - 17ms/step
Epoch 6/1000
4/4 - 0s - loss: 1.3175 - val_loss: 1.2351 - 72ms/epoch - 18ms/step
Epoch 7/1000
4/4 - 0s - loss: 1.2833 - val_loss: 1.2087 - 70ms/epoch - 17ms/step
Epoch 8/1000
4/4 - 0s - loss: 1.2539 - val_loss: 1.1844 - 73ms/epoch - 18ms/step
Epoch 9/1000
4/4 - 0s - loss: 1.2264 - val_loss: 1.1618 - 68ms/epoch - 17ms/step
Epoch 10/1000
4/4 - 0s - loss: 1.1998 - val_loss: 1.1423 - 55ms/epoch - 14ms/step
Epoch 11/1000
4/4 - 0s - loss: 1.1778 - val_loss: 1.1246 - 50ms/epoch - 12ms/step
Epoch 12/1000
4/4 - 0s - loss: 1.1576 - val_loss: 1.1090 - 67ms/epoch - 17ms/step
Epoch 13/1000
4/4 - 0s - l

Now that the neural network is trained, we can make predictions about the test set. The following code predicts the type of iris for test set and displays the first five irises.

In [ ]:
pred = model.predict(x_test)
print(pred[0:5]) # print first five predictions

2/2 [==============================] - 0s 5ms/step
[[1.36420920e-01 7.49744177e-01 1.13834865e-01]
 [9.90822196e-01 9.17118229e-03 6.48552259e-06]
 [7.44056478e-02 3.22757423e-01 6.02836967e-01]
 [1.32445559e-01 6.97811663e-01 1.69742659e-01]
 [1.35248244e-01 7.30962217e-01 1.33789420e-01]]


Each line provides the probability that the iris is one of the 3 types of iris in the data set.

### Saving best weights


It would be good idea to keep track of the most optimal weights during the entire training operation.

An additional monitor, ModelCheckpoint, is used and saves a copy of the neural network to best_weights.hdf5 each time the validation score of the neural network improves.

Once training is done, we just reload this file and we have the optimal training weights that were found.

In [ ]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "/content/drive/MyDrive/Colab Notebooks/CSC219-MachineLearning/labs/data"

filename = os.path.join(path,"iris.csv")
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")
x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

checkpointer = ModelCheckpoint(filepath="dnn/best_weights.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)

model.load_weights('dnn/best_weights.hdf5') # load weights from best model

Epoch 1/1000
4/4 - 2s - loss: 1.6049 - val_loss: 1.4825 - 2s/epoch - 577ms/step
Epoch 2/1000
4/4 - 0s - loss: 1.5123 - val_loss: 1.4010 - 96ms/epoch - 24ms/step
Epoch 3/1000


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 - 0s - loss: 1.4245 - val_loss: 1.3250 - 105ms/epoch - 26ms/step
Epoch 4/1000
4/4 - 0s - loss: 1.3453 - val_loss: 1.2544 - 98ms/epoch - 25ms/step
Epoch 5/1000
4/4 - 0s - loss: 1.2718 - val_loss: 1.1896 - 117ms/epoch - 29ms/step
Epoch 6/1000
4/4 - 0s - loss: 1.2054 - val_loss: 1.1305 - 110ms/epoch - 28ms/step
Epoch 7/1000
4/4 - 0s - loss: 1.1450 - val_loss: 1.0800 - 106ms/epoch - 26ms/step
Epoch 8/1000
4/4 - 0s - loss: 1.0971 - val_loss: 1.0496 - 97ms/epoch - 24ms/step
Epoch 9/1000
4/4 - 0s - loss: 1.0737 - val_loss: 1.0339 - 97ms/epoch - 24ms/step
Epoch 10/1000
4/4 - 0s - loss: 1.0568 - val_loss: 1.0191 - 102ms/epoch - 25ms/step
Epoch 11/1000
4/4 - 0s - loss: 1.0431 - val_loss: 1.0058 - 138ms/epoch - 35ms/step
Epoch 12/1000
4/4 - 0s - loss: 1.0307 - val_loss: 0.9936 - 163ms/epoch - 41ms/step
Epoch 13/1000
4/4 - 0s - loss: 1.0198 - val_loss: 0.9808 - 51ms/epoch - 13ms/step
Epoch 14/1000
4/4 - 0s - loss: 1.0084 - val_loss: 0.9687 - 54ms/epoch - 14ms/step
Epoch 15/1000
4/4 - 0s - loss

### Potential Keras Issue on Small Networks Regarding Saving Optimal Weights

You might occasionally see this error:

```
OSError: Unable to create file (Unable to open file: name = 'dnn/best_weights.hdf5', errno = 22, error message = 'invalid argument', flags = 13, o_flags = 302)
```

Usually you can just run rerun the code and it goes away.  This is an unfortnuate result of saving a file each time the validation score improves (as described in the previous section).  If the errors improve two rapidly, you might try to save the file twice and get an error from these two saves overlapping.  For larger neural networks this will not be a problem because each training step will take longer, allowing for plenty of time for the previous save to complete.   

## Evaluating Classification Models

### (1) Calculate Classification Accuracy/Precision/Recall/F1-Score

By default, Keras will return the predicted probability for each class. We can change these prediction probabilities into the actual iris predicted with **argmax**.

In [ ]:
pred = model.predict(x_test)
pred

2/2 [==============================] - 0s 8ms/step


array([[4.84938035e-03, 8.80813420e-01, 1.14337079e-01],
       [9.92564619e-01, 7.43530504e-03, 7.40192352e-11],
       [8.33976173e-07, 2.31531542e-02, 9.76845980e-01],
       [3.85425868e-03, 8.46795917e-01, 1.49349943e-01],
       [2.58757407e-03, 8.70149314e-01, 1.27263173e-01],
       [9.85245585e-01, 1.47543913e-02, 8.03163092e-10],
       [3.99660170e-02, 9.52488184e-01, 7.54572777e-03],
       [6.45358232e-05, 2.12420508e-01, 7.87514925e-01],
       [4.04394552e-04, 3.44933480e-01, 6.54662073e-01],
       [1.99533589e-02, 9.58530188e-01, 2.15165466e-02],
       [3.19570216e-04, 4.22556758e-01, 5.77123761e-01],
       [9.86557543e-01, 1.34424381e-02, 1.73565706e-09],
       [9.95103359e-01, 4.89650341e-03, 3.78874987e-11],
       [9.86750543e-01, 1.32494969e-02, 1.39863421e-09],
       [9.94556308e-01, 5.44369593e-03, 6.67971303e-11],
       [5.07090101e-03, 9.28924859e-01, 6.60041943e-02],
       [4.20417155e-06, 3.55025865e-02, 9.64493155e-01],
       [1.66354161e-02, 9.42910

In [ ]:
pred = np.argmax(pred,axis=1) # raw probabilities to choose class (highest probability)
print(pred)

[1 0 2 1 1 0 1 2 2 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


Now that we have the actual iris flower predicted, we can calculate the percent accuracy (how many were correctly classified).

In [ ]:
y_true= np.argmax(y_test,axis=1)

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

Accuracy score: 0.9736842105263158


In [ ]:
score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

Precision score: 0.9757085020242916


In [ ]:
score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

Recall score: 0.9736842105263158


In [ ]:
score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

F1 score: 0.9735839598997494


In [ ]:
print(metrics.classification_report(y_true, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      0.91      0.95        11
           2       0.92      1.00      0.96        12

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38



### (2) Calculate Classification Cross-Entropy Loss (Log Loss)  

Log loss is an error metric that is often used in place of accuracy for classification.  

Log loss allows for "partial credit". For example, a model might be used to classify A, B and C.  The correct answer might be A, however if the classification network chose B as having the highest probability, then accuracy gives the neural network no credit for this classification.  

However, with log loss, the probability of the correct answer is added to the score.  For example, the correct answer might be A, but if the neural network only predicted .4 probability of A being correct, then the value -log(.4) is added.

$$ logloss = -\frac{1}{N}\sum^N_{i=1}\sum^M_{j=1}y_{ij} \log(\hat{y}_{ij}) $$

The following code shows the logloss scores that correspond to the average probablity for the correct item. The **pred** column specifies the average robability for the correct class.  The **logloss** column specifies the log loss for that probability.


calculating loss

In [ ]:
# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
print(pred[0:5])
print()
print("y_test:")
print(y_test[0:5])

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

2/2 [==============================] - 0s 4ms/step
Numpy array of predictions
[[4.8493803e-03 8.8081342e-01 1.1433708e-01]
 [9.9256462e-01 7.4353050e-03 7.4019235e-11]
 [8.3397617e-07 2.3153154e-02 9.7684598e-01]
 [3.8542587e-03 8.4679592e-01 1.4934994e-01]
 [2.5875741e-03 8.7014931e-01 1.2726317e-01]]

y_test:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
Log loss score: 0.1180629165063872


## Evaluating Regression Models

Regression results are evaluated differently than classification. Consider the following code.

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import metrics

path = "/content/drive/MyDrive/Colab Notebooks/CSC219-MachineLearning/labs/data"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')

encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.25, random_state=45)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10))
model.add(Dense(10))
model.add(Dense(10))

model.add(Dense(1))  # 1 output neuron


model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.fit(x_train,y_train, validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

<ipython-input-17-d6ebaee71354>:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('name',1,inplace=True)


Epoch 1/1000
10/10 - 2s - loss: 244736.3438 - val_loss: 50261.8945 - 2s/epoch - 192ms/step
Epoch 2/1000
10/10 - 0s - loss: 13747.9102 - val_loss: 7330.2275 - 94ms/epoch - 9ms/step
Epoch 3/1000
10/10 - 0s - loss: 13467.7979 - val_loss: 11058.1846 - 139ms/epoch - 14ms/step
Epoch 4/1000
10/10 - 0s - loss: 5322.9282 - val_loss: 608.0283 - 107ms/epoch - 11ms/step
Epoch 5/1000
10/10 - 0s - loss: 1239.9482 - val_loss: 1834.8124 - 72ms/epoch - 7ms/step
Epoch 6/1000
10/10 - 0s - loss: 1483.4537 - val_loss: 704.9721 - 92ms/epoch - 9ms/step
Epoch 7/1000
10/10 - 0s - loss: 700.9958 - val_loss: 621.8273 - 78ms/epoch - 8ms/step
Epoch 8/1000
10/10 - 0s - loss: 727.2465 - val_loss: 549.4726 - 64ms/epoch - 6ms/step
Epoch 9/1000
10/10 - 0s - loss: 647.6301 - val_loss: 533.3834 - 51ms/epoch - 5ms/step
Epoch 10/1000
10/10 - 0s - loss: 627.2051 - val_loss: 515.6107 - 68ms/epoch - 7ms/step
Epoch 11/1000
10/10 - 0s - loss: 614.8265 - val_loss: 505.0542 - 58ms/epoch - 6ms/step
Epoch 12/1000
10/10 - 0s - loss:

### Mean Square Error

The mean square error is the sum of the squared differences between the prediction ($\hat{y}$) and the expected ($y$).  MSE values are not of a particular unit.  If an MSE value has decreased for a model, that is good. Low MSE values are desired.

$ \text{MSE} = \frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2 $


In [ ]:
# Predict
pred = model.predict(x_test)

# Measure MSE error.
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

4/4 [==============================] - 0s 3ms/step
Final score (MSE): 49.28150939941406


### Root Mean Square Error

The root mean square (RMSE) is essentially the square root of the MSE.  Because of this, the RMSE error is in the same units as the training data outcome. Low RMSE values are desired.

$ \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2} $

In [ ]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 7.020079135894775


# Performance Improvement by Normalizing Features and Tuning Hyperparameters


There are many different settings that you can use for a neural network.  These can affect performance.  The following code changes some of these, beyond their default values:

* **activation:** relu, sigmoid, tanh
* **Layers and Neuron Counts**
* **optimizer:** adam, sgd, rmsprop, and [others](https://keras.io/optimizers/)

In [ ]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "/content/drive/MyDrive/Colab Notebooks/CSC219-MachineLearning/labs/data"
preprocess = True

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# create feature vector
missing_median(df, 'horsepower')
encode_text_dummy(df, 'origin')
df.drop('name',1,inplace=True)

if preprocess:
    encode_numeric_zscore(df, 'horsepower')
    encode_numeric_zscore(df, 'weight')
    encode_numeric_zscore(df, 'cylinders')
    encode_numeric_zscore(df, 'displacement')
    encode_numeric_zscore(df, 'acceleration')
    encode_numeric_zscore(df, 'year')

# Encode to a 2D matrix for training
x,y = to_xy(df,'mpg')

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

<ipython-input-23-a3cb074e385d>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('name',1,inplace=True)


Epoch 1/1000
10/10 - 2s - loss: 613.5257 - val_loss: 566.4623 - 2s/epoch - 163ms/step
Epoch 2/1000
10/10 - 0s - loss: 583.2334 - val_loss: 533.9932 - 91ms/epoch - 9ms/step
Epoch 3/1000
10/10 - 0s - loss: 541.8113 - val_loss: 479.3809 - 82ms/epoch - 8ms/step
Epoch 4/1000
10/10 - 0s - loss: 471.5322 - val_loss: 391.1856 - 78ms/epoch - 8ms/step
Epoch 5/1000
10/10 - 0s - loss: 363.8999 - val_loss: 261.3913 - 88ms/epoch - 9ms/step
Epoch 6/1000
10/10 - 0s - loss: 213.2343 - val_loss: 114.5524 - 82ms/epoch - 8ms/step
Epoch 7/1000
10/10 - 0s - loss: 74.6083 - val_loss: 31.7939 - 76ms/epoch - 8ms/step
Epoch 8/1000
10/10 - 0s - loss: 32.3393 - val_loss: 38.1170 - 104ms/epoch - 10ms/step
Epoch 9/1000
10/10 - 0s - loss: 30.5549 - val_loss: 23.7904 - 84ms/epoch - 8ms/step
Epoch 10/1000
10/10 - 0s - loss: 21.8137 - val_loss: 18.6904 - 90ms/epoch - 9ms/step
Epoch 11/1000
10/10 - 0s - loss: 19.4701 - val_loss: 16.2668 - 56ms/epoch - 6ms/step
Epoch 12/1000
10/10 - 0s - loss: 17.0978 - val_loss: 14.8866

In [ ]:
# Predict and measure RMSE
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

3/3 [==============================] - 0s 4ms/step
Score (RMSE): 2.2074921131134033


In [ ]:
# print out prediction
df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(pred, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

,ground_truth,predicted
0,33.0,33.622807
1,28.0,30.883892
2,19.0,20.726608
3,13.0,15.191343
4,14.0,13.122478
...,...,...
75,19.9,19.715271
76,17.5,17.474388
77,28.0,30.366440
78,29.0,30.144081
